In [1]:
import io
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px
warnings.filterwarnings('ignore')

%matplotlib inline

import polars as pl
from skimpy import skim
from summarytools import dfSummary
#summarytools doesnt work with polars library, it works with pandas tho

#%load_ext cudf.pandas
#supercharges workflow with GPU acceleration using cudf.pandas
#Im having pip install issues with cudf so commenting it out for now

In [2]:
fifa17 = pd.read_csv('fifa_final17.csv')
fifa18 = pd.read_csv('fifa_final18.csv')
fifa19 = pd.read_csv('fifa_final19.csv')
fifa20 = pd.read_csv('fifa_final20.csv')
fifa21 = pd.read_csv('fifa_final21.csv')
fifa22 = pd.read_csv('fifa_final22.csv')
fifa23 = pd.read_csv('fifa_final23.csv')

In [3]:
# Merge all CSVs in the folder into a single polars DataFrame

import glob

# Load all CSV files in the directory
csv_files = glob.glob("*.csv")
print(f"Found CSV files: {csv_files}")

datasets = {}
for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file)
        datasets[csv_file] = df
        print(f"\nLoaded {csv_file}:")
        print(f"Shape: {df.shape}")
        print(f"Columns: {list(df.columns)}")
    except Exception as e:
        print(f"Error loading {csv_file}: {e}")

warnings.filterwarnings("ignore")

Found CSV files: ['fifa_final17.csv', 'fifa_final18.csv', 'fifa_final19.csv', 'fifa_final20.csv', 'fifa_final21.csv', 'fifa_final22.csv', 'fifa_final23.csv']

Loaded fifa_final17.csv:
Shape: (17560, 61)
Columns: ['ID', 'Name', 'Age', 'Nationality', 'Overall', 'Potential', 'Club', 'Value', 'Wage', 'Special', 'Preferred Foot', 'International Reputation', 'Weak Foot', 'Skill Moves', 'Real Face', 'Jersey Number', 'Joined', 'Contract Valid Until', 'Height', 'Weight', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes', 'Best Position', 'Best Overall Rating', 'Position_clean', 'Body Type Cle

In [4]:
#Standardization

# FIFA 2023 uses Kit Number
fifa23.rename(columns={'Kit Number': 'Jersey Number'}, inplace=True)

fifa17['Release Clause'] = np.nan
fifa17['DefensiveAwareness'] = np.nan
fifa18['DefensiveAwareness'] = np.nan
fifa19['DefensiveAwareness'] = np.nan

numeric_columns = [
    'Age', 'Overall', 'Potential', 'International Reputation', 
    'Weak Foot', 'Skill Moves', 'Jersey Number', 'Height', 'Weight'
]

for col in numeric_columns:
    if col in fifa17.columns:
        fifa17[col] = pd.to_numeric(fifa17[col], errors='coerce')

for col in numeric_columns:
    if col in fifa18.columns:
        fifa18[col] = pd.to_numeric(fifa18[col], errors='coerce')

for col in numeric_columns:
    if col in fifa19.columns:
        fifa19[col] = pd.to_numeric(fifa19[col], errors='coerce')

for col in numeric_columns:
    if col in fifa20.columns:
        fifa20[col] = pd.to_numeric(fifa20[col], errors='coerce')

for col in numeric_columns:
    if col in fifa21.columns:
        fifa21[col] = pd.to_numeric(fifa21[col], errors='coerce')

for col in numeric_columns:
    if col in fifa22.columns:
        fifa22[col] = pd.to_numeric(fifa22[col], errors='coerce')

for col in numeric_columns:
    if col in fifa23.columns:
        fifa23[col] = pd.to_numeric(fifa23[col], errors='coerce')

In [5]:
def merge_fifa_datasets_for_ml():
    """
    Merge FIFA datasets (2017-2022) for ML wage prediction with full attributes.
    FIFA 2023 kept separate as test/validation set.
    """
    
    # File paths
    fifa_files = {
        2017: 'fifa_final17.csv',
        2018: 'fifa_final18.csv', 
        2019: 'fifa_final19.csv',
        2020: 'fifa_final20.csv',
        2021: 'fifa_final21.csv',
        2022: 'fifa_final22.csv',
        2023: 'fifa_final23.csv'  # Will be kept separate
    }
    
    # Training data (FIFA 17-22) - Full attributes
    training_datasets = []
    
    print("Loading FIFA datasets for training (2017-2022)...")
    
    for year in range(2017, 2023):
        try:
            df = pd.read_csv(fifa_files[year])
            df['Year'] = year
            print(f"FIFA {year}: {df.shape[0]:,} players, {df.shape[1]} columns")
            
            # Standardize critical columns for ML
            standardized_df = standardize_columns(df, year)
            training_datasets.append(standardized_df)
            
        except FileNotFoundError:
            print(f"Warning: {fifa_files[year]} not found, skipping...")
            continue
    
    # Merge training datasets (FIFA 17-22)
    print("\nMerging training datasets...")
    fifa_training = pd.concat(training_datasets, ignore_index=True, sort=False)
    
    # Load FIFA 23 separately for test/validation
    print("\nLoading FIFA 2023 for test/validation...")
    try:
        fifa_2023 = pd.read_csv(fifa_files[2023])
        fifa_2023['Year'] = 2023
        fifa_2023 = standardize_columns(fifa_2023, 2023)
        print(f"FIFA 2023: {fifa_2023.shape[0]:,} players, {fifa_2023.shape[1]} columns")
    except FileNotFoundError:
        print("Warning: FIFA 2023 file not found")
        fifa_2023 = None
    
    # Clean datasets for ML
    print("\nCleaning datasets for ML...")
    fifa_training_clean = clean_for_ml(fifa_training)
    fifa_2023_clean = clean_for_ml(fifa_2023) if fifa_2023 is not None else None
    
    # Feature engineering for ML
    print("\nEngineering features for ML...")
    fifa_training_final = engineer_ml_features(fifa_training_clean)
    fifa_2023_final = engineer_ml_features(fifa_2023_clean) if fifa_2023_clean is not None else None
    
    # Save datasets
    print("\nSaving datasets...")
    fifa_training_final.to_csv('fifa_training_2017_2022.csv', index=False)
    print(f"Training dataset saved: {fifa_training_final.shape[0]:,} samples, {fifa_training_final.shape[1]} features")
    
    if fifa_2023_final is not None:
        fifa_2023_final.to_csv('fifa_test_2023.csv', index=False)
        print(f"Test dataset saved: {fifa_2023_final.shape[0]:,} samples, {fifa_2023_final.shape[1]} features")
    
    # Dataset summary
    print_dataset_summary(fifa_training_final, fifa_2023_final)
    
    return fifa_training_final, fifa_2023_final

def standardize_columns(df, year):
    """Standardize column names and handle year-specific differences"""
    df = df.copy()
    
    # Standardize column names
    column_mapping = {
        'Kit Number': 'Jersey Number',  # FIFA 2023 uses Kit Number
    }
    
    df.rename(columns=column_mapping, inplace=True)
    
    # Add missing columns with appropriate defaults for ML
    if year == 2017:
        df['Release Clause'] = np.nan  # Not available in FIFA 17
    
    if year <= 2019:
        df['DefensiveAwareness'] = np.nan  # Added in FIFA 20
    
    # Ensure consistent data types
    numeric_columns = [
        'Age', 'Overall', 'Potential', 'International Reputation', 
        'Weak Foot', 'Skill Moves', 'Jersey Number', 'Height', 'Weight'
    ]
    
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    return df

def clean_for_ml(df):
    """Clean dataset specifically for ML model training"""
    if df is None:
        return None
    
    df = df.copy()
    
    print(f"  Original dataset: {df.shape[0]:,} players")
    
    # Remove players with missing wages (target variable)
    df = df.dropna(subset=['Wage'])
    print(f"  After removing missing wages: {df.shape[0]:,} players")
    
    # Clean wage column - remove currency symbols and convert to numeric
    df['Wage_Clean'] = clean_wage_column(df['Wage'])
    df = df.dropna(subset=['Wage_Clean'])
    print(f"  After cleaning wages: {df.shape[0]:,} players")
    
    # Remove players with 0 wage (likely non-playing staff)
    df = df[df['Wage_Clean'] > 0]
    print(f"  After removing 0 wages: {df.shape[0]:,} players")
    
    # Clean value column similarly
    df['Value_Clean'] = clean_value_column(df['Value'])
    
    # Remove players missing critical features
    critical_features = ['Overall', 'Age', 'Position_clean']
    df = df.dropna(subset=critical_features)
    print(f"  After removing missing critical features: {df.shape[0]:,} players")
    
    return df

def clean_wage_column(wage_series):
    """Convert wage strings to numeric values in thousands"""
    def parse_wage(wage_str):
        if pd.isna(wage_str) or wage_str == '':
            return np.nan
        
        # Remove currency symbols and clean
        wage_str = str(wage_str).replace('€', '').replace('$', '').replace(',', '').strip()
        
        try:
            if 'K' in wage_str:
                return float(wage_str.replace('K', ''))
            elif 'M' in wage_str:
                return float(wage_str.replace('M', '')) * 1000
            else:
                # Assume it's already in thousands or convert from base currency
                num = float(wage_str)
                if num > 10000:  # Likely in base currency, convert to thousands
                    return num / 1000
                return num
        except:
            return np.nan
    
    return wage_series.apply(parse_wage)

def clean_value_column(value_series):
    """Convert value strings to numeric values in millions"""
    def parse_value(value_str):
        if pd.isna(value_str) or value_str == '':
            return np.nan
        
        # Remove currency symbols and clean
        value_str = str(value_str).replace('€', '').replace('$', '').replace(',', '').strip()
        
        try:
            if 'K' in value_str:
                return float(value_str.replace('K', '')) / 1000  # Convert to millions
            elif 'M' in value_str:
                return float(value_str.replace('M', ''))
            else:
                # Assume it's in base currency
                num = float(value_str)
                return num / 1000000  # Convert to millions
        except:
            return np.nan
    
    return value_series.apply(parse_value)

def engineer_ml_features(df):
    """Create engineered features for ML model"""
    if df is None:
        return None
    
    df = df.copy()
    
    # Age-based features
    df['Age_Category'] = pd.cut(df['Age'], 
    bins=[0, 21, 25, 29, 33, 50], 
    labels=['Youth', 'Young', 'Prime', 'Veteran', 'Elder'])
    
    # Potential vs Overall gap
    df['Potential_Gap'] = df['Potential'] - df['Overall']
    df['Potential_Ratio'] = df['Overall'] / df['Potential']
    
    # Contract years remaining (handle string values)
    if 'Contract Valid Until' in df.columns:
        # Convert contract year to numeric, handling various formats
        df['Contract_Year_Clean'] = pd.to_numeric(df['Contract Valid Until'], errors='coerce')
        current_year = df['Year']
        df['Contract_Years_Remaining'] = df['Contract_Year_Clean'] - current_year
        df['Contract_Years_Remaining'] = df['Contract_Years_Remaining'].clip(lower=0, upper=10).fillna(0)
    else:
        df['Contract_Years_Remaining'] = 0
    
    # Position grouping for ML
    position_mapping = {
        'GK': 'Goalkeeper',
        'CB': 'Defender', 'LB': 'Defender', 'RB': 'Defender', 'LWB': 'Defender', 'RWB': 'Defender',
        'CDM': 'Midfielder', 'CM': 'Midfielder', 'CAM': 'Midfielder', 'LM': 'Midfielder', 'RM': 'Midfielder',
        'LW': 'Forward', 'RW': 'Forward', 'CF': 'Forward', 'ST': 'Forward'
    }
    df['Position_Group'] = df['Position_clean'].map(position_mapping).fillna('Other')
    
    # Create skill composites based on position (handle missing columns gracefully)
    skill_columns = ['Finishing', 'ShotPower', 'LongShots', 'ShortPassing', 'LongPassing', 'Vision',
                    'Marking', 'StandingTackle', 'SlidingTackle', 'Strength', 'Stamina', 'Jumping',
                    'Dribbling', 'BallControl', 'Curve', 'Acceleration', 'SprintSpeed']
    
    # Only create composites if the skill columns exist
    if all(col in df.columns for col in ['Finishing', 'ShotPower', 'LongShots']):
        df['Attacking_Composite'] = (df['Finishing'] + df['ShotPower'] + df['LongShots']).fillna(0) / 3
    else:
        df['Attacking_Composite'] = 0
        
    if all(col in df.columns for col in ['ShortPassing', 'LongPassing', 'Vision']):
        df['Passing_Composite'] = (df['ShortPassing'] + df['LongPassing'] + df['Vision']).fillna(0) / 3
    else:
        df['Passing_Composite'] = 0
        
    if all(col in df.columns for col in ['Marking', 'StandingTackle', 'SlidingTackle']):
        df['Defending_Composite'] = (df['Marking'] + df['StandingTackle'] + df['SlidingTackle']).fillna(0) / 3
    else:
        df['Defending_Composite'] = 0
        
    if all(col in df.columns for col in ['Strength', 'Stamina', 'Jumping']):
        df['Physical_Composite'] = (df['Strength'] + df['Stamina'] + df['Jumping']).fillna(0) / 3
    else:
        df['Physical_Composite'] = 0
        
    if all(col in df.columns for col in ['Dribbling', 'BallControl', 'Curve']):
        df['Technical_Composite'] = (df['Dribbling'] + df['BallControl'] + df['Curve']).fillna(0) / 3
    else:
        df['Technical_Composite'] = 0
        
    if all(col in df.columns for col in ['Acceleration', 'SprintSpeed']):
        df['Pace_Composite'] = (df['Acceleration'] + df['SprintSpeed']).fillna(0) / 2
    else:
        df['Pace_Composite'] = 0
    
    # International reputation categories
    if 'International Reputation' in df.columns:
        df['Int_Rep_Category'] = pd.cut(df['International Reputation'], 
        bins=[0, 1, 2, 3, 4, 5], 
        labels=['Unknown', 'Local', 'National', 'Continental', 'Worldwide'])
    else:
        df['Int_Rep_Category'] = 'Unknown'
    
    # Market tier based on value
    if 'Value_Clean' in df.columns:
        df['Market_Tier'] = pd.cut(df['Value_Clean'], 
        bins=[0, 1, 5, 20, 50, 200], 
        labels=['Budget', 'Standard', 'Premium', 'Star', 'Superstar'])
    
    return df

def print_dataset_summary(train_df, test_df=None):
    """Print summary statistics for the datasets"""
    print("\n" + "="*60)
    print("DATASET SUMMARY FOR ML WAGE PREDICTION")
    print("="*60)
    
    print(f"\nTRAINING SET (FIFA 2017-2022):")
    print(f"  Total samples: {train_df.shape[0]:,}")
    print(f"  Total features: {train_df.shape[1]:,}")
    print(f"  Years covered: {sorted(train_df['Year'].unique())}")
    print(f"  Wage range: €{train_df['Wage_Clean'].min():.0f}K - €{train_df['Wage_Clean'].max():.0f}K")
    print(f"  Average wage: €{train_df['Wage_Clean'].mean():.0f}K")
    
    if test_df is not None:
        print(f"\nTEST SET (FIFA 2023):")
        print(f"  Total samples: {test_df.shape[0]:,}")
        print(f"  Total features: {test_df.shape[1]:,}")
        print(f"  Note: Limited attributes available for FIFA 2023")
    
    print(f"\nKEY FEATURES FOR ML MODEL:")
    print(f"  Target variable: Wage_Clean")
    print(f"  Main predictors: Overall, Age, Position_Group, Value_Clean")
    print(f"  Engineered features: Age_Category, Potential_Gap, Skill Composites")
    print(f"  Categorical features: Nationality, Club, Position_clean")
    
    print(f"\nPOSITION DISTRIBUTION:")
    print(train_df['Position_Group'].value_counts())
    
    print(f"\nDATA QUALITY:")
    missing_pct = (train_df.isnull().sum() / len(train_df) * 100).round(1)
    high_missing = missing_pct[missing_pct > 10].sort_values(ascending=False)
    if len(high_missing) > 0:
        print(f"  Features with >10% missing values:")
        for col, pct in high_missing.head(10).items():
            print(f"    {col}: {pct}%")
    else:
        print(f"  No features with >10% missing values")
    
    print("\n" + "="*60)
    print("Ready for EDA and ML Model Training!")
    print("="*60)

# Run the merger
if __name__ == "__main__":
    train_data, test_data = merge_fifa_datasets_for_ml()

Loading FIFA datasets for training (2017-2022)...
FIFA 2017: 17,560 players, 62 columns
FIFA 2018: 17,927 players, 63 columns
FIFA 2019: 17,943 players, 63 columns
FIFA 2020: 17,104 players, 64 columns
FIFA 2021: 17,108 players, 64 columns
FIFA 2022: 16,710 players, 64 columns

Merging training datasets...

Loading FIFA 2023 for test/validation...
FIFA 2023: 17,660 players, 28 columns

Cleaning datasets for ML...
  Original dataset: 104,352 players
  After removing missing wages: 104,352 players
  After cleaning wages: 104,352 players
  After removing 0 wages: 102,496 players
  After removing missing critical features: 102,496 players
  Original dataset: 17,660 players
  After removing missing wages: 17,660 players
  After cleaning wages: 17,660 players
  After removing 0 wages: 17,390 players
  After removing missing critical features: 17,389 players

Engineering features for ML...

Saving datasets...
Training dataset saved: 102,496 samples, 80 features
Test dataset saved: 17,389 samp

In [6]:
"""The AI Code did the following
I started with standardization partially, then gave the task to AI, it did it plus added more stuff
It made functions to clean and make the data in wage and value to numeric
I already did that in my previous eda iterations
I kept it so that the whole Ai generated code goes through smoothly
Only major thing AI helped was feature engineering function
And in merging 7 csvs using loops which made the code much cleaner and readable
I have to learn more about feature engineering and merging csvs
"""

'The AI Code did the following\nI started with standardization partially, then gave the task to AI, it did it plus added more stuff\nIt made functions to clean and make the data in wage and value to numeric\nI already did that in my previous eda iterations\nI kept it so that the whole Ai generated code goes through smoothly\nOnly major thing AI helped was feature engineering function\nAnd in merging 7 csvs using loops which made the code much cleaner and readable\nI have to learn more about feature engineering and merging csvs\n'